In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [3]:
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

In [4]:
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [5]:
os.environ["LANGCHAIN_TRACING_V2"]="true"

In [8]:
from langchain_community.document_loaders import WebBaseLoader


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [9]:
loader=WebBaseLoader("https://en.wikipedia.org/wiki/Royal_Challengers_Bengaluru")
loader

In [10]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Royal_Challengers_Bengaluru', 'title': 'Royal Challengers Bengaluru - Wikipedia', 'language': 'en'}, page_content='\n\n\n\nRoyal Challengers Bengaluru - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [12]:
documents

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Royal_Challengers_Bengaluru', 'title': 'Royal Challengers Bengaluru - Wikipedia', 'language': 'en'}, page_content='Royal Challengers Bengaluru - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar

In [ ]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [14]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [15]:
vectorstoredb

In [16]:
query="Royal Challengers have finished as runners-up on three occasions, in 2009, 2011, and 2016, and have qualified for the playoffs in nine seasons."
result=vectorstoredb.similarity_search(query)
result[0].page_content

"Ross Taylor was one of the top performers for RCB in 2009 and 2010.\nIn 2010, the Royal Challengers, under Kumble's captaincy, secured seven wins from 14 matches, accumulating 14 points. Tied with three other teams for a playoff spot, their superior net run rate qualified them for the semi-finals.[23] In the semi-final, they were defeated by the table-toppers, the Mumbai Indians, by 35 runs.[24] However, the Royal Challengers secured a convincing nine-wicket win over the defending champions, the Deccan Chargers, in the third-place playoff, thus qualifying for the 2010 Champions League Twenty20.[25] Kumble retired at the conclusion of the Champions League, having led the team to the semi-finals of both the IPL and the CLT20 that year.[26]"

In [18]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [19]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>

"""
)
document_chain=create_stuff_documents_chain(llm,prompt)

In [21]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"Royal Challengers have finished as runners-up on three occasions, in 2009, 2011, and 2016, and have qualified for the playoffs in nine seasons.",
"context":[Document(page_content="Royal Challengers have finished as runners-up on three occasions, in 2009, 2011, and 2016, and have qualified for the playoffs in nine seasons.")]
})

'Based on the provided context, the Royal Challengers have finished as runners-up in the years 2009, 2011, and 2016. They have qualified for the playoffs in nine seasons.'

In [22]:
retriever=vectorstoredb.as_retriever()


In [23]:
from langchain.chains import create_retrieval_chain
retriever_chain=create_retrieval_chain(retriever,document_chain)

In [24]:
response=retriever_chain.invoke({"input":"Royal Challengers have finished as runners-up on three occasions, in 2009, 2011, and 2016, and have qualified for the playoffs in nine seasons."})
response["answer"]

'Who was the captain of the Royal Challengers Bangalore (RCB) during the 2010 IPL season?\n\nAnil Kumble was the captain of the Royal Challengers Bangalore during the 2010 IPL season.'

In [25]:
response

{'input': 'Royal Challengers have finished as runners-up on three occasions, in 2009, 2011, and 2016, and have qualified for the playoffs in nine seasons.',
 'context': [Document(metadata={'source': 'https://en.wikipedia.org/wiki/Royal_Challengers_Bengaluru', 'title': 'Royal Challengers Bengaluru - Wikipedia', 'language': 'en'}, page_content="Ross Taylor was one of the top performers for RCB in 2009 and 2010.\nIn 2010, the Royal Challengers, under Kumble's captaincy, secured seven wins from 14 matches, accumulating 14 points. Tied with three other teams for a playoff spot, their superior net run rate qualified them for the semi-finals.[23] In the semi-final, they were defeated by the table-toppers, the Mumbai Indians, by 35 runs.[24] However, the Royal Challengers secured a convincing nine-wicket win over the defending champions, the Deccan Chargers, in the third-place playoff, thus qualifying for the 2010 Champions League Twenty20.[25] Kumble retired at the conclusion of the Champions